# SVD compression

In order to understand the truncating function of SVD a bit better, we will have a look at the its application on image data. We treat the image as a matrix and apply the SVD to this data.

Import of some useful modules

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

Load an image using PIL and convert it to an black and white image.

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
import os
from os.path import join
image_list=[(f,join('images',f)) for f in os.listdir('images')]
filename="joshua.jpg"
def set_filename(image):
    global filename
    filename=image
interact(set_filename,image=image_list);

In [ ]:
img=Image.open(filename).convert('L')
img_np=np.asarray(img)
#Plot the original image in grayscale
fig, axes=plt.subplots()
axes.imshow(img_np,cmap=plt.cm.gray);

Apply the SVD to the image, truncate it and display the result. Make sure that the largest singular values are taken into account when you truncate.

In [ ]:
trunc=20
#Apply the SVD to the array
u,s,vh=np.linalg.svd(img_np)
reconstruction=u[:,:trunc]@np.diag(s[:trunc])@vh[:trunc,:]
#Plot the reconstruction with reduced number of singular values
fig,axes=plt.subplots()
axes.imshow(reconstruction,cmap=plt.cm.gray);

We use the interactive possibilities of Jupyter Notebooks to explore the interplay between image quality and the number of singular values that are taken into account.

In [ ]:
fig,axes=plt.subplots()
contents=axes.imshow(img, cmap=plt.cm.gray)

def update_image_truncated(dim):                                                                                                                                                                                                                                                           
    reconstruction=u[:,:dim]@np.diag(s[:dim])@vh[:dim,:]
    contents.set_data(reconstruction)
interact(update_image_truncated,dim=IntSlider(min=2,max=512,continuous_update=False));